# User 리뷰 형태소 분석
## 1. Colab에 Mecab 설치

In [ ]:
from google.colab import drive # 구글 드라이브 마운트 작업
drive.mount('/content/drive')

In [ ]:
# Colab에 Mecab 설치
!pip install konlpy
!pip install mecab-python
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)


## 2.유저 리뷰 데이터 전처리

In [ ]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from collections import Counter
from konlpy.tag import Mecab
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

### 1) 데이터 로드


In [ ]:
# import warnings
# import pandas as pd
# import chardet

# warnings.simplefilter(action='ignore', category=pd.errors.ParserWarning)



# # 감지된 인코딩을 사용하여 파일을 읽습니다
# total_data = pd.read_csv("/content/통합유저.csv", encoding='latin1', on_bad_lines='skip')


In [ ]:
import pandas as pd
total_data = pd.read_excel('/content/drive/MyDrive/통합유저_전처리완.xlsx')


In [ ]:
total_data[:5]

### 2) 훈련 데이터 & 테스트 데이터 분리

In [ ]:
total_data[:5]

In [ ]:
# 중복 제거

total_data.drop_duplicates(subset=['place_review'],inplace=True)

print('총 샘플의 수 :',len(total_data))

In [ ]:
print(total_data.isnull().values.any())

In [ ]:
# 훈련 데이터와 테스트 데이터를 3:1 비율로 분리
from sklearn.model_selection import train_test_split
train_data , test_data = train_test_split(total_data,test_size=0.25,random_state =43)

print('훈련용 리뷰의 개수 :', len(train_data))
print('테스트용 리뷰의 개수 :', len(test_data))

### 3) 데이터 정제

In [ ]:
# 한글과 공백을 제외하고 모두 제거
import numpy as np
train_data['place_review'] = train_data['place_review'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train_data['place_review'].replace('', np.nan, inplace=True)
print(train_data.isnull().sum())

In [ ]:
test_data.drop_duplicates(subset = ['place_review'], inplace=True) # 중복 제거
test_data['place_review'] = test_data['place_review'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
test_data['place_review'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test_data = test_data.dropna(how='any') # Null 값 제거
print('전처리 후 테스트용 샘플의 개수 :',len(test_data))

### 5) 토큰화

형태소 분석기 Mecab을 사용하여 토큰화 작업을 수행합니다. 다음은 임의의 문장에 대해서 테스트한 토큰화 결과입니다.

In [ ]:
# Colab에 Mecab 설치
!pip install konlpy
!pip install mecab-python
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)


In [ ]:
import re
import urllib.request
from collections import Counter
from konlpy.tag import Mecab
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
mecab = Mecab()
print(mecab.morphs('와 이런 것도 상품이라고 차라리 내가 만드는 게 나을 뻔'))

In [ ]:
# 불용어 지정해서 필요없는 토큰 제거

stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']

In [ ]:
train_data['tokenized'] = train_data['place_review'].apply(mecab.morphs)
train_data['tokenized'] = train_data['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])



In [ ]:
test_data['tokenized'] = test_data['place_review'].apply(mecab.morphs)
test_data['tokenized'] = test_data['tokenized'].apply(lambda x : [item for item in x if item not in stopwords])

### 6) 단어와 길이 분포 확인
긍정 리뷰에는 주로 어떤 단어들이 많이 등장하고, 부정 리뷰에는 주로 어떤 단어들이 등장하는지 두 가지 경우에 대해서 각 단어의 빈도수를 계산해보겠습니다. 각 레이블에 따라서 별도로 단어들의 리스트를 저장해줍니다.

In [ ]:
# 긍정과 부정리뷰 구분
# np.hstack() 함수는 주어진 배열들을 수평(가로)으로 합치는 기능

# negative_words = np.hstack(train_data[train_data.label == 0]['tokenized'].values)
# positive_words = np.hstack(train_data[train_data.label == 1]['tokenized'].values)

In [ ]:
# Counter()를 사용하여 각 단어에 대한 빈도수를 카운트

# negative_word_count = Counter(negative_words)
# print(negative_word_count.most_common(22))

In [ ]:
# positive_word_count = Counter(positive_words)
# print(positive_word_count.most_common(20))

In [ ]:
# fig,(ax1,ax2) = plt.subplots(1,2,figsize=(10,5))
# text_length = train_data[train_data['label']==1]['tokenized'].map(lambda x: len(x))
# ax1.hist(text_length, color='red')
# ax1.set_title('Positive Reviews')
# ax1.set_xlabel('length of samples')
# ax1.set_ylabel('number of samples')
# print('긍정 리뷰의 평균 길이 :', np.mean(text_length))

# text_length = train_data[train_data['label']==0]['tokenized'].map(lambda x : len(x))
# ax2.hist(text_length, color='blue')
# ax2.set_title('Negative Reviews')
# fig.suptitle('Words in texts')
# ax2.set_xlabel('length of samples')
# ax2.set_ylabel('number of samples')
# print('부정 리뷰의 평균 길이 :', np.mean(text_length))
# plt.show()

In [ ]:
X_train = train_data['tokenized'].values
# y_train = train_data['label'].values
X_test= test_data['tokenized'].values
# y_test = test_data['label'].values

In [ ]:
X_train

### 7) 정수 인코딩

기계가 텍스트를 숫자로 처리할 수 있도록 훈련 데이터와 테스트 데이터에 정수 인코딩을 수행

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [ ]:
# 각 단어에 고유한 정수 부여 -> tokenizer.word_index 를 통해 확인 가능

# 1번 등장하는 단어는 배제

threshold = 2
total_cnt = len(tokenizer.word_index) # 단어수
rare_cnt = 0 # 등장 빈도수가 threshold 보다 작은 단어의 개수를 카운트
total_freq = 0  # 훈련 데이터의 전체 단어 빈도수 총 함
rare_freq = 0 # 등장 빈도수가 threshold 보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍을 key 와 value 로 받는다.
# 단어 : 빈도 로 나옴
for key,value in tokenizer.word_counts.items() :
    total_freq = total_freq + value
    # 단어의 등장 빈도수가 threshold보다 작으면 :
    if value < threshold :
        rare_cnt = rare_cnt +1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어는 28393개 등장. 등장 빈도가 1번인

단어는 전체에서 40% 차지.

실제 훈련 데이터에서 회귀 단어의 등장 비율은 약 0.8% --> 자연어 처리에서 중요하지 않을것으로 판단. 제외시힘

In [ ]:
# 전체 단어 개수 중 빈도수 2이하인 단어 개수는 제거.

# 0번 패딩 토큰과 1번 OOV 토큰을 고려하여 +2

vocab_size = total_cnt - rare_cnt + 2
print('단어 집합의 크기 :',vocab_size)

In [ ]:
tokenizer = Tokenizer(vocab_size, oov_token = 'OOV')
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

# oov_token: 사전에 없는 단어(Out-Of-Vocabulary, OOV)가 등장할 때 대체할 토큰을 지정합니다. oov_token='OOV'로 설정하면, 사전에 없는 단어는 모두 'OOV'로 대체됩니다.
# 이렇게 하면 모델이 예측할 때 사전에 없는 단어를 처리할 수 있습니다.

In [ ]:
print(X_train[:3])

In [ ]:
print(X_test[:3])

### 8) 패딩
서로 다른 길이의 샘플들의 길이를 동일하게 맞춰주는 패딩

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
print('리뷰의 최대 길이 :',max(len(review) for review in X_train))
print('리뷰의 평균 길이 :',sum(map(len, X_train))/len(X_train))
plt.hist([len(review) for review in X_train], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

리뷰의 최대 길이는 168, 평균 길이는 약 31

In [ ]:
def below_threshold_len(max_len,nested_list):
    count = 0
    for sentence in nested_list :
        if len(sentence) <= max_len:
            count = count+ 1
    print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (count / len(nested_list))*100))

In [ ]:
max_len = 168
below_threshold_len(max_len, X_train)

In [ ]:
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

## 3. GRU로 네이버 쇼핑 리뷰 감성 분류

### GRU

시계열 데이터나 순차 데이터를 다루기 위해 설계

- GRu의 구조는 업데이트 게이트와 리셋 게이트

    1) 업데이트 게이트는 이전 상태를 얼마나 유지하고 새로운 입력을 얼마나 반영할지 결정

    2) 리셋 게이트는 이전 상태를 얼마나 무시할 지 결정

    3) 새로운 메모리 내용
        - 리셋 게이트의 결과를 사용해 새로운 후보 활성 상태를 계산
    4) 최종 활성 상태
        - 업데이트 게이트와 새로운 메모리 내용을 사용해서 최종 상태를 계

- GRU의 특징

    1) 간단한 구조: LSTM보다 게이트 수가 적고, 계산 복잡도가 낮아 학습과 예측이 더 빠릅니다.
    
    2) 효율적인 학습: 적은 수의 게이트로도 장기 의존성을 효과적으로 학습할 수 있습니다.
    
    3) 성능: 많은 경우에서 LSTM과 유사한 성능을 보이며, 특히 데이터 양이 적거나 모델이 간단해야 할 때 유리합니다.

하이퍼파라미터인 임베팅 벡터의 차원은 100, 은닉층은 128.

모델은 다대일 구조의 LSTM을 사용 -> 두 개의 선택지 중 하나를 예측하는 이진분류 수행

이진분류의 경우에는 활성화함수는 시그모이드, 손실함수는 크로스 엔트로피 함수 사용

하이퍼파라미터인 배치크기는 64, 15에포크 수행

EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)

-> 검증 데이터 손실이 증가하면 과적합 징후이므로 검증 데이터손실이 4회 증가하면 학습을 조기종료

ModelCheckpoint : 검증데이터의 정확도가 이전보다 좋아질 경우에만 모델을 저장

validation_split=0.2을 사용하여 훈련 데이터의 20%를 검증 데이터로 분리해서 사용하고, 검증 데이터를 통해서 훈련이 적절히 되고 있는지 확인



1. 입력 처리

    - 텍스트 데이터를 단어 또는 토큰 단위로 나누어 시퀀스로 변환합니다.
    - 각 단어를 임베딩 벡터로 변환하여 GRU의 입력으로 사용합니다.

2. GRU 레이어

    - GRU는 입력 시퀀스를 순차적으로 처리하며, 각 시점에서 새로운 입력과 이전 상태를 기반으로 새로운 후보 활성 상태를 계산합니다.
    - 이 활성 상태는 GRU 내부의 정보 저장소로, 문맥 정보를 담고 있습니다.

3. 새로운 후보 활성 상태

    - 새로운 후보 활성 상태 는 현재 시점에서 입력된 단어와 이전 시점의 정보를 조합하여 계산됩니다.
    
    - 이 활성 상태는 단순히 하나의 단어가 긍정인지 부정인지를 의미하지 않습니다. 대신, 현재까지 입력된 모든 단어의 정보를 종합한 문맥 정보를 나타냅니다.
4. 최종 활성 상태

    - 각 시점의 활성 상태를 업데이트한 후, 마지막 시점의 활성 상태는 전체 문장의 문맥을 반영합니다.
    - 이 최종 상태를 사용하여 문장의 감성을 예측합니다.
5. 출력 레이어

    - 최종 활성 상태를 사용하여 감성(긍정 또는 부정)을 예측합니다. 일반적으로, 이 과정은 하나 이상의 밀집층(Dense Layer)과 소프트맥스(Softmax) 또는 시그모이드(Sigmoid) 활성 함수로 이루어집니다.

In [ ]:
from tensorflow.keras.layers import Embedding, Dense, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

embedding_dim = 100
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size,embedding_dim))
model.add(GRU(hidden_units))
model.add(Dense(1,activation = 'sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)

mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])

history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=64, validation_split=0.2)

In [ ]:
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

## 4. 리뷰 예측

In [ ]:
def sentiment_predict(new_sentence):
  new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
  new_sentence = mecab.morphs(new_sentence)
  new_sentence = [word for word in new_sentence if not word in stopwords]
  encoded = tokenizer.texts_to_sequences([new_sentence])
  pad_new = pad_sequences(encoded, maxlen = max_len)

  score = float(loaded_model.predict(pad_new))
  if(score > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.".format((1 - score) * 100))

In [ ]:
sentiment_predict('이 상품 진짜 좋아요... 저는 강추합니다. 대박')

In [ ]:
sentiment_predict('진짜 배송도 늦고 개짜증나네요. 뭐 이런 걸 상품이라고 만듬?')

In [ ]:
sentiment_predict('판매자님... 너무 짱이에요.. 대박나삼')

In [ ]:
sentiment_predict('ㅁㄴㅇㄻㄴㅇㄻㄴㅇ리뷰쓰기도 귀찮아')